<a href="https://colab.research.google.com/github/JordanHanotiaux/Internship/blob/main/Benchmark_YOLOE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
!pip install ultralytics

!pip install supervision

In [1]:
!wget https://huggingface.co/GLIPModel/GLIP/resolve/main/mdetr_annotations/final_flickr_separateGT_train.json \
    -P /content/annotations/

!wget https://huggingface.co/GLIPModel/GLIP/resolve/main/mdetr_annotations/final_mixed_train_no_coco.json \
    -P /content/annotations/

!wget https://raw.githubusercontent.com/ultralytics/ultralytics/main/ultralytics/cfg/datasets/Objects365.yaml \
    -P /content/annotations/

--2025-07-09 10:18:49--  https://huggingface.co/GLIPModel/GLIP/resolve/main/mdetr_annotations/final_flickr_separateGT_train.json
Resolving huggingface.co (huggingface.co)... 3.165.102.58, 3.165.102.6, 3.165.102.22, ...
Connecting to huggingface.co (huggingface.co)|3.165.102.58|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.hf.co/repos/13/50/135006ff86825f748a869e2da849789ef2c4adf60ca9f4b959caa837f55c5082/00805253492afd77c066b7132612f8c5fb6a4d0f8bff6d593cbfec23e1aac173?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27final_flickr_separateGT_train.json%3B+filename%3D%22final_flickr_separateGT_train.json%22%3B&response-content-type=application%2Fjson&Expires=1752059929&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc1MjA1OTkyOX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy8xMy81MC8xMzUwMDZmZjg2ODI1Zjc0OGE4NjllMmRhODQ5Nzg5ZWYyYzRhZGY2MGNhOWY0Yjk1OWNhYTgzN2Y1NWM1MDgyLzAwODA1MjUzN

In [ ]:
import os
import zipfile
import json
import re
import yaml
import requests
import torch
import numpy as np
import supervision as sv
import matplotlib.pyplot as plt

from PIL import Image
from io import BytesIO
from google.colab import drive
from ultralytics import YOLOE
from tqdm import tqdm
from ultralytics import YOLOE
from sklearn.metrics import average_precision_score
from collections import defaultdict

In [ ]:
# Import Lvis minival dataset.

!rm -rf lvis_minival_only.json
drive.mount('/content/drive')

with open("/content/drive/MyDrive/minival.txt", "r") as f:
    minival_ids = set(int(line.strip().split('/')[-1].split('.')[0]) for line in f)

images = []
annotations = []
categories = []
category_ids_in_minival = set()

# Extract and filter LVIS data from the ZIP archive
with zipfile.ZipFile("/content/drive/MyDrive/lvis_v1_val.json.zip", "r") as zf:
    with zf.open("lvis_v1_val.json", "r") as f:
        lvis_data = json.load(f)

    for img in lvis_data["images"]:
        if img["id"] in minival_ids:
            images.append(img)

    for ann in lvis_data["annotations"]:
        if ann["image_id"] in minival_ids:
            annotations.append(ann)
            category_ids_in_minival.add(ann['category_id'])

    for cat in lvis_data["categories"]:
        if cat['id'] in category_ids_in_minival:
            categories.append(cat)

# Reconstruct filtered LVIS structure
lvis_data = {
    "images": images,
    "annotations": annotations,
    "categories": categories
}

# Save to JSON
with open("lvis_minival_only.json", "w") as f:
    json.dump(lvis_data, f)

print(f"Images : {len(images)}")
print(f"Annotations : {len(annotations)}")
print(f"Categories : {len(categories)}")

In [ ]:
# Create categories's set of Lvis dataset

def clean_category(name):
    # Delete pattern as _(xxx)
    name = re.sub(r'_\([^)]*\)', '', name)
    # Replace remaining underscores by spaces
    name = re.sub(r'_', ' ', name)
    return name.strip()

lvis_categories = set()
for cat in lvis_val['categories']:
    cleaned = clean_category(cat['name'])
    lvis_categories.add(cleaned)

print("Nombre de catégories (LVIS dataset):", len(lvis_categories))

In [ ]:
# Import Object365 dataset

with open('/content/annotations/Objects365.yaml') as f:
    O365_train = yaml.safe_load(f)

object365_categories_processed = set()
for v in O365_train['names'].values():
    for word in v.split('/'):
        object365_categories_processed.add(word.lower().strip())

print("Nombre de catégories (Objects365 dataset):", len(object365_categories_processed))

# Calcul des catégories communes et uniques
seen_categories = lvis_categories.intersection(object365_categories_processed)
print("Nombre de catégories communes (LVIS-Object365) :", len(seen_categories))

unseen_categories = lvis_categories.difference(object365_categories_processed)
print("Nombre de catégories non communes (LVIS-Object365) :", len(unseen_categories))

In [ ]:
# Import GoldG dataset

def import_json_file(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    return data

flickr_train = import_json_file('/content/annotations/final_flickr_separateGT_train.json')
gqa_train = import_json_file('/content/annotations/final_mixed_train_no_coco.json')

# Info dataset
print(" Flickr")
print(" - Nombre d'images :", len(flickr_train['images']))
print(" - Nombre d'annotations :", len(flickr_train['annotations']))

print("\n GQA")
print(" - Nombre d'images :", len(gqa_train['images']))
print(" - Nombre d'annotations :", len(gqa_train['annotations']))

# Extract annotated phrases (grounding categories) from captions
def extract_grounding_phrases(dataset):
    categories = set()
    for data in dataset:
      annotations = data['annotations']
      ann_idx = 0
      for img in data['images']:
          caption = img['caption'].lower()
          image_id = img['id']
          while ann_idx < len(annotations) and annotations[ann_idx]['image_id'] == image_id:
              for interval in annotations[ann_idx]['tokens_positive']:
                  phrase = caption[interval[0]:interval[1]]
                  categories.add(phrase.strip())
              ann_idx += 1
          ann_idx += 1
    return categories

grounding_categories = extract_grounding_phrases([flickr_train, gqa_train])

print("\n Résumé")
print(f" - Nombre de catégories : {len(grounding_categories)}")
print(f" - Exemples : {list(grounding_categories)[:5]}")

# Match unseen categories with annotated phrases (grounding categories)
example = 0
for category in unseen_categories.copy():
    for phrase in grounding_categories:
      if all(word in phrase.split() for word in category.split()):
        if example < 5:
          print("Catégorie trouvée :", category)
          print("Phrase trouvée :", phrase)
          example += 1
        unseen_categories.discard(category)
        seen_categories.add(category)
        break

print("Nombre de catégories communes(LVIS-Object 365-GoldG dataset):", len(seen_categories))
print("Nombre de catégories non communes (LVIS-Object 365-GoldG dataset):", len(unseen_categories))

In [ ]:
# Predict AP50 score for each category across the LVIs minival dataset

def get_category_mapping(data):
  """
    Creates a mapping from category ID to category name.

    Args:
        data (dict): JSON Dataset.

    Returns:
        dict: Mapping from category ID (int) to category name (str).
    """
    return {cat['id']: cat['name'] for cat in data['categories']}

def get_annotations_by_image(data, image_id):
  """
    Retrieves all annotations associated with a specific image.

    Args:
        data (dict): JSON Dataset.
        image_id (int): ID of the image.

    Returns:
        list: List of annotation dicts for the specified image.
    """
    return [ann for ann in data['annotations'] if ann['image_id'] == image_id]

def group_boxes_by_category(annotations, id_to_name):
  """
    Groups bounding boxes by their category names.

    Args:
        annotations (list): List of annotation dicts for a single image.
        id_to_name (dict): Mapping from category ID to category name.

    Returns:
        dict: Dictionary mapping category name to a list of bounding boxes.
    """
    category_to_boxes = defaultdict(list)
    for ann in annotations:
        category_name = id_to_name[ann['category_id']]
        category_to_boxes[category_name].append(ann['bbox'])
    return category_to_boxes

def compute_iou(boxA, boxB):
  """
    Computes the Intersection over Union (IoU) between two bounding boxes.

    Args:
        boxA (list): First box in XYXY format [x1, y1, x2, y2].
        boxB (list): Second box in XYXY format [x1, y1, x2, y2].

    Returns:
        float: IoU score between the two boxes.
    """
    ix1 = max(boxA[0], boxB[0])
    iy1 = max(boxA[1], boxB[1])
    ix2 = min(boxA[2], boxB[2])
    iy2 = min(boxA[3], boxB[3])
    inter = max(ix2 - ix1, 0) * max(iy2 - iy1, 0)

    areaA = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    areaB = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])
    union = areaA + areaB - inter
    return inter / union if union > 0 else 0

def convert_bbox_xywh_to_xyxy(bbox):
   """
    Converts bounding box from [x, y, width, height] to [x1, y1, x2, y2].

    Args:
        bbox (list): Bounding box in XYWH format.

    Returns:
        list: Bounding box in XYXY format.
    """
    x, y, w, h = bbox
    return [x, y, x + w, y + h]

def compute_ap(gt_boxes, pred_boxes, pred_scores, iou_threshold=0.5):
  """
    Computes Average Precision (AP) at a given IoU threshold for a single category.

    Args:
        gt_boxes (list): Ground-truth bounding boxes in XYWH format.
        pred_boxes (list): Predicted boxes in XYXY format.
        pred_scores (list): Confidence scores for each predicted box.
        iou_threshold (float): IoU threshold to consider a prediction correct.

    Returns:
        float: Average Precision (AP) score.
    """
    if not pred_boxes:
        return 0.0

    ious = []
    for pred_box in pred_boxes:
        iou_max = 0
        for gt_box in gt_boxes:
            iou = compute_iou(pred_box, convert_bbox_xywh_to_xyxy(gt_box))
            iou_max = max(iou_max, iou)
        ious.append(iou_max)

    y_true = [1 if iou >= iou_threshold else 0 for iou in ious]
    return average_precision_score(y_true, pred_scores) if any(y_true) else 0.0

def process_image(image, data, model, id_to_name, thresholds, ap50_scores):
  """
    Runs inference on a single image, computes AP scores across all thresholds
    for each ground-truth category, and updates the AP score dictionary.

    Args:
        image (dict): Image metadata dict from JSON dataset.
        data (dict): JSON dataset.
        model (YOLOE): Initialized detection model.
        id_to_name (dict): Category ID to name mapping.
        thresholds (list): List of confidence thresholds to evaluate.
        ap50_scores (defaultdict): Nested dict to store AP scores.
    """
    response = requests.get(image['coco_url'])
    img = Image.open(BytesIO(response.content)).convert("RGB")

    annotations = get_annotations_by_image(data, image['id'])
    category_to_boxes = group_boxes_by_category(annotations, id_to_name)
    categories = list(category_to_boxes.keys())

    model.set_classes(categories, model.get_text_pe(categories))
    result = model.predict(img, conf=0.001, verbose=False)[0]
    detections = sv.Detections.from_ultralytics(result)

    for category_name, gt_boxes in category_to_boxes.items():
        for threshold in thresholds:
            pred_boxes = []
            pred_scores = []

            for label, score, box in zip(detections['class_name'], detections.confidence, detections.xyxy):
                if label == category_name and score >= threshold:
                    pred_boxes.append(box)
                    pred_scores.append(score)

            ap = compute_ap(gt_boxes, pred_boxes, pred_scores)
            ap50_scores[category_name][threshold].append(ap)

def evaluate_dataset(data, model_path, max_images=5):
  """
    Evaluates a detection model on a subset of the JSON dataset by computing
    AP@0.5 scores per category across multiple confidence thresholds.

    Args:
        data (dict): JSON dataset.
        model_path (str): Path to the pretrained YOLOE model.
        max_images (int): Maximum number of images to evaluate.

    Returns:
        dict: Nested dictionary {category_name: {threshold: [AP scores]}}.
    """
    ap50_scores = defaultdict(lambda: defaultdict(list))
    id_to_name = get_category_mapping(data)
    thresholds = [round(x, 1) for x in np.arange(0.0, 1.01, 0.1)]

    for image in tqdm(data['images'][:max_images]):
        model = YOLOE(model_path)
        # model.to("cuda")
        process_image(image, data, model, id_to_name, thresholds, ap50_scores)

    return ap50_scores

ap50_scores = evaluate_dataset(lvis_val, "pretrain/yoloe-v8l-seg.pt", max_images=5)

In [ ]:
# Computes the mean Average Precision (mAP@0.5) across all categories for each confidence threshold,
# then plots the mAP@0.5 curve.

mAP50_per_threshold = {}
for threshold in thresholds:
    all_aps_at_threshold = []
    for category_name in ap50_scores:
        if ap50_scores[category_name][threshold]:
            all_aps_at_threshold.extend(ap50_scores[category_name][threshold])
    if all_aps_at_threshold:
        mAP50_per_threshold[threshold] = np.mean(all_aps_at_threshold)
    else:
        mAP50_per_threshold[threshold] = 0.0

thresholds_sorted = sorted(mAP50_per_threshold.keys())
mAP50_values = [mAP50_per_threshold[t] for t in thresholds_sorted]

plt.figure(figsize=(10, 6))
plt.plot(thresholds_sorted, mAP50_values, marker='o')
plt.xlabel('Confidence Threshold')
plt.ylabel('mAP@0.5')
plt.title('mAP@0.5 vs. Confidence Threshold')
plt.grid(True)
plt.show()

print("mAP@0.5 per Threshold:")
for threshold, mAP50 in mAP50_per_threshold.items():
    print(f"Threshold: {threshold:.1f}, mAP@0.5: {mAP50:.4f}")

In [ ]:
rare_categories = set()
common_categories = set()
frequent_categories = set()

for cat in lvis_val['categories']:
  freq = cat['frequency']
  if freq == 'r':
    rare_categories.add(cat['name'])
  elif freq == 'c':
    common_categories.add(cat['name'])
  elif freq == 'f':
    frequent_categories.add(cat['name'])

print(f"Catégories rares: {len(rare_categories)}")
print(f"Catégories communes: {len(common_categories)}")
print(f"Catégories fréquentes: {len(frequent_categories)}")

In [ ]:
def split_ap50_scores_by_frequency(ap50_scores, frequency_dict):
    """
    Splits AP50 scores into separate dictionaries based on category frequency.

    Args:
        ap50_scores (dict): Nested dict {category_name: {threshold: [ap_scores]}}.
        frequency_dict (dict): Dict with keys 'rare', 'common', 'frequent', each mapping to a set of category names.

    Returns:
        Tuple of defaultdicts: (rare_scores, common_scores, frequent_scores)
    """
    rare_scores = defaultdict(lambda: defaultdict(list))
    common_scores = defaultdict(lambda: defaultdict(list))
    frequent_scores = defaultdict(lambda: defaultdict(list))

    for category, thresholds_data in ap50_scores.items():
        if category in frequency_dict['rare']:
            target = rare_scores
        elif category in frequency_dict['common']:
            target = common_scores
        elif category in frequency_dict['frequent']:
            target = frequent_scores
        else:
            continue  # skip categories not in any group

        for threshold, aps in thresholds_data.items():
            target[category][threshold].extend(aps)

    return rare_scores, common_scores, frequent_scores


def compute_mAP50_per_threshold(scores_by_category, thresholds):
    """
    Computes mAP@0.5 per threshold across multiple categories.

    Args:
        scores_by_category (dict): Nested dict {category_name: {threshold: [ap_scores]}}.
        thresholds (list): List of thresholds to evaluate.

    Returns:
        dict: Mapping from threshold to averaged mAP@0.5 score.
    """
    mAP50_per_threshold = {}

    for threshold in thresholds:
        all_scores = []
        for category_scores in scores_by_category.values():
            all_scores.extend(category_scores.get(threshold, []))

        mAP50_per_threshold[threshold] = (
            np.mean(all_scores) if all_scores else 0.0
        )

    return mAP50_per_threshold


def plot_mAP_curves(mAP50_dicts, thresholds, labels):
    """
    Plots mAP@0.5 curves for different category frequency groups.

    Args:
        mAP50_dicts (list of dict): List of threshold->mAP50 dicts.
        thresholds (list): List of thresholds used.
        labels (list): Corresponding labels for each curve.
    """
    plt.figure(figsize=(10, 6))
    for mAP50, label in zip(mAP50_dicts, labels):
        sorted_thresh = sorted(mAP50.keys())
        sorted_mAP = [mAP50[t] for t in sorted_thresh]
        plt.plot(sorted_thresh, sorted_mAP, marker='o', label=label)

    plt.xlabel('Confidence Threshold')
    plt.ylabel('mAP@0.5')
    plt.title('mAP@0.5 vs. Confidence Threshold per Category Frequency')
    plt.grid(True)
    plt.legend()
    plt.show()


def print_mAP50(mAP50_per_threshold, label):
    """
    Prints the mAP@0.5 for each threshold.

    Args:
        mAP50_per_threshold (dict): Mapping threshold -> mAP50.
        label (str): Label describing the category group.
    """
    print(f"\nmAP@0.5 per Threshold for {label} Categories:")
    for threshold, score in mAP50_per_threshold.items():
        print(f"Threshold: {threshold:.1f}, mAP@0.5: {score:.4f}")

category_frequencies = {
    'rare': rare_categories,
    'common': common_categories,
    'frequent': frequent_categories
}

rare_scores, common_scores, frequent_scores = split_ap50_scores_by_frequency(
    ap50_scores, category_frequencies
)

rare_mAP50 = compute_mAP50_per_threshold(rare_scores, thresholds)
common_mAP50 = compute_mAP50_per_threshold(common_scores, thresholds)
frequent_mAP50 = compute_mAP50_per_threshold(frequent_scores, thresholds)

plot_mAP_curves(
    [rare_mAP50, common_mAP50, frequent_mAP50],
    thresholds,
    labels=['Rare', 'Common', 'Frequent']
)

print_mAP50(rare_mAP50, 'Rare')
print_mAP50(common_mAP50, 'Common')
print_mAP50(frequent_mAP50, 'Frequent')



In [ ]:
def categorize_categories_by_object_size(lvis_data, id_to_name, small_thresh=32*32, large_thresh=96*96):
    """
    Categorizes object categories into small, normal, and large based on average bounding box area.

    Args:
        lvis_data (dict): LVIS annotations data.
        id_to_name (dict): Mapping from category ID to name.
        small_thresh (int): Area threshold under which objects are considered small.
        large_thresh (int): Area threshold above which objects are considered large.

    Returns:
        dict: A dictionary with keys 'small', 'normal', 'large' each containing a set of category names.
    """
    small, normal, large = set(), set(), set()

    for ann in lvis_data['annotations']:
        area = ann['bbox'][2] * ann['bbox'][3]  # width * height
        category_name = id_to_name[ann['category_id']]
        if area < small_thresh:
            small.add(category_name)
        elif area > large_thresh:
            large.add(category_name)
        else:
            normal.add(category_name)

    # Ensure mutual exclusivity
    normal -= small | large
    small -= normal | large
    large -= small | normal

    return {'small': small, 'normal': normal, 'large': large}


def split_ap50_scores_by_object_size(ap50_scores, size_categories):
    """
    Splits AP50 scores by object size categories.

    Args:
        ap50_scores (dict): Nested dict {category_name: {threshold: [ap_scores]}}.
        size_categories (dict): Dict with keys 'small', 'normal', 'large', each a set of category names.

    Returns:
        Tuple of defaultdicts: (small_scores, normal_scores, large_scores)
    """
    small_scores = defaultdict(lambda: defaultdict(list))
    normal_scores = defaultdict(lambda: defaultdict(list))
    large_scores = defaultdict(lambda: defaultdict(list))

    for category, thresholds_data in ap50_scores.items():
        if category in size_categories['small']:
            target = small_scores
        elif category in size_categories['normal']:
            target = normal_scores
        elif category in size_categories['large']:
            target = large_scores
        else:
            continue

        for threshold, aps in thresholds_data.items():
            target[category][threshold].extend(aps)

    return small_scores, normal_scores, large_scores

# Step 1: Categorize by object size
object_size_categories = categorize_categories_by_object_size(lvis_val, id_to_name)

# Step 2: Split AP50 scores by size category
small_scores, normal_scores, large_scores = split_ap50_scores_by_object_size(ap50_scores, object_size_categories)

# Step 3: Compute mAP@0.5 curves
small_mAP50 = compute_mAP50_per_threshold(small_scores, thresholds)
normal_mAP50 = compute_mAP50_per_threshold(normal_scores, thresholds)
large_mAP50 = compute_mAP50_per_threshold(large_scores, thresholds)

# Step 4: Plot
plot_mAP_curves(
    [small_mAP50, normal_mAP50, large_mAP50],
    thresholds,
    labels=['Small Objects', 'Normal Objects', 'Large Objects']
)

# Step 5: Print results
print_mAP50(small_mAP50, 'Small Objects')
print_mAP50(normal_mAP50, 'Normal Objects')
print_mAP50(large_mAP50, 'Large Objects')



In [ ]:
def split_ap50_scores_by_seen_status(ap50_scores, seen_categories, unseen_categories, clean_fn):
    """
    Splits AP50 scores into seen and unseen categories based on cleaned names.

    Args:
        ap50_scores (dict): Nested dict {category_name: {threshold: [ap_scores]}}.
        seen_categories (set): Set of cleaned names of seen categories.
        unseen_categories (set): Set of cleaned names of unseen categories.
        clean_fn (function): Function to clean/normalize category names.

    Returns:
        Tuple of defaultdicts: (seen_scores, unseen_scores)
    """
    seen_scores = defaultdict(lambda: defaultdict(list))
    unseen_scores = defaultdict(lambda: defaultdict(list))

    for category, thresholds_data in ap50_scores.items():
        clean_name = clean_fn(category)
        if clean_name in unseen_categories:
            target = unseen_scores
        elif clean_name in seen_categories:
            target = seen_scores
        else:
            continue

        for threshold, aps in thresholds_data.items():
            target[category][threshold].extend(aps)

    return seen_scores, unseen_scores

# Step 1: Split scores by seen/unseen
seen_scores, unseen_scores = split_ap50_scores_by_seen_status(
    ap50_scores, seen_categories, unseen_categories, clean_category
)

# Step 2: Compute mAP curves
seen_mAP50 = compute_mAP50_per_threshold(seen_scores, thresholds)
unseen_mAP50 = compute_mAP50_per_threshold(unseen_scores, thresholds)

# Step 3: Plot
plot_mAP_curves(
    [unseen_mAP50, seen_mAP50],
    thresholds,
    labels=['Unseen Objects', 'Seen Objects']
)

# Step 4: Print
print_mAP50(seen_mAP50, 'Seen Objects')
print_mAP50(unseen_mAP50, 'Unseen Objects')
